## Parametrize Charges example

#### Import basic packages

In [ ]:
import os
# Gets the absolute path of the current notebook (works reliably if run inside the notebook itself)
notebook_path = os.path.abspath("")
# Get the parent directory (one level up)
parent_dir = os.path.abspath(os.path.join(notebook_path, ".."))
# Change working directory to the parent directory
os.chdir(parent_dir)
import numpy as np
import pandas as pd
import eeco.costs as costs
from eeco.costs import (
    DEMAND, ENERGY, PEAK, HALF_PEAK, OFF_PEAK, SUPER_OFF_PEAK
)
# If this fails, run "pip install -e . " in the main directory of eeco

#### Import a tariff .csv file

In [ ]:
path_to_tariffsheet = "eeco/eeco/data/tariff.csv"
# path_to_tariffsheet = "../eeco/tests/data/input/billing_pge.csv"
rate_df = pd.read_csv(path_to_tariffsheet, sep=",")


In [ ]:
rate_df.head()

## Specify charge variants as a list of dictionaries
### Each with entries for:
- 'percent_change_dict': nested dictionary for charge scaling by period type
- 'scale_all_demand': global scaling for all demand charges
- 'scale_all_energy': global scaling for all energy charges  
- 'shift_peak_hours_before': hours to shift peak window start (must be multiple of 0.25)
- 'shift_peak_hours_after': hours to shift peak window end (must be multiple of 0.25)

The example below specifies 4 tariff variants as 4 entries in the list

In [ ]:
# Each dictionary specifies a variant. See docstrings for all options.
variants = [
    # Double peak charges
    {
        'percent_change_dict': {
            DEMAND: {PEAK: 2.0, HALF_PEAK: 2.0, OFF_PEAK: 1.0, SUPER_OFF_PEAK: 1.0},
            ENERGY: {PEAK: 2.0, HALF_PEAK: 2.0, OFF_PEAK: 1.0, SUPER_OFF_PEAK: 1.0}
        },
        'variant_name': 'double_peak_and_half_peak'
    },
    # Increse peak charges from specific keys
    {
        "percent_change_dict": {
            "electric_demand_peak-summer": 2.2,
            "electric_demand_half_peak-summer": 2.2,
        },
        'variant_name': 'increase_peak_demand_2.2_manual'
    },
    # Halve peak charges
    {
        'percent_change_dict': {
            DEMAND: {PEAK: 0.5, HALF_PEAK: 0.5, OFF_PEAK: 1.0, SUPER_OFF_PEAK: 1.0},
            ENERGY: {PEAK: 0.5, HALF_PEAK: 0.5, OFF_PEAK: 1.0, SUPER_OFF_PEAK: 1.0}
        },
        'variant_name': 'decrease_peak_and_half_peak'
    },
    # Increase all demand/energy charges
    {
        'percent_change_dict': {
            DEMAND: 1.15,
            ENERGY: 1.15
        },
        'variant_name': 'increase_all'
    },
    # Expand the peak window (shift earlier start, later end)
    {
        'shift_peak_hours_before': -1.5,  # negative = earlier start (must be multiple of 0.25)
        'shift_peak_hours_after': 1.5,    # positive = later end (must be multiple of 0.25)
        'variant_name': 'expand_peak_window'
    }
]

### Specify date range to generate parametrized tariffs


In [ ]:
# 1 week including 07-01 (00:00) through 07-05 (23:45)
start_dt = pd.Timestamp('2023-07-01')
end_dt = pd.Timestamp('2023-07-05')

### Generate new and baseline charge dictionaries

In [ ]:
# Baseline tariff dict
base_tariff_dict = costs.get_charge_dict(start_dt=start_dt, end_dt=end_dt, rate_data=rate_df)

# Parametrized variants
new_tariff_dicts = costs.parametrize_charge_dict(
    start_dt=start_dt,
    end_dt=end_dt,
    rate_data=rate_df,
    variants=variants
)
print(new_tariff_dicts.keys())

### Plot the energy and demand charges to visualize the parametrization

In [ ]:
# make sure matplotlib is in your python environment before running this section
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
colors = plt.cm.jet(np.linspace(0, 1, len(new_tariff_dicts)))
colors[0] = (0, 0, 0, 1)

max_demand_charge = 0
max_energy_charge = 0

# Create time index
time_index = pd.date_range(start=start_dt, end=end_dt - pd.Timedelta(minutes=15), freq='15min')

for (variant_key, tariff_dict), color in zip(new_tariff_dicts.items(), colors):
    # Initialize arrays with the correct length
    sum_energy_array = np.zeros(len(time_index))
    sum_demand_array = np.zeros(len(time_index))
    
    for charge_key, charge_array in tariff_dict.items():
        # Skip scalar values (like customer charges)
        if not isinstance(charge_array, np.ndarray):
            continue
            
        # Only process arrays that match our time index length
        if len(charge_array) == len(time_index):
            if 'electric_energy' in charge_key:
                sum_energy_array += charge_array
            elif 'electric_demand' in charge_key:
                sum_demand_array += charge_array
    
    if variant_key == 'original':
        ax1.plot(time_index, sum_energy_array, color=color, label=variant_key, linewidth=4)
        ax2.plot(time_index, sum_demand_array, color=color, linewidth=4)
    else:
        ax1.plot(time_index, sum_energy_array, color=color, label=variant_key)
        ax2.plot(time_index, sum_demand_array, color=color)
   
    max_demand_charge = max(max_demand_charge, sum_demand_array.max())
    max_energy_charge = max(max_energy_charge, sum_energy_array.max())

# Format x-axis
for ax in [ax1, ax2]:
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
    ax.xaxis.set_minor_locator(mdates.HourLocator(interval=6))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

ax1.set_ylim(0.0, 1.1*max_energy_charge)
ax2.set_ylim(0.0, 1.1*max_demand_charge)
ax1.set_ylabel('SUM of energy charges ($/kWh)')
ax2.set_ylabel('SUM of demand charges ($/kW)')
ax1.set_title('Energy variants')
ax2.set_title('Demand variants')

fig.legend(bbox_to_anchor=(1.01, 0.5), loc='center left', borderaxespad=0.)
plt.tight_layout()
plt.show()